In [19]:
import pandas as pd 
import numpy as np

In [26]:
import json
import pandas as pd

# Load JSON
with open('./Item_data/items.json', 'r') as f:
    data = json.load(f)

# Loop through all elements of the JSON
for category, tests in data.items():
    print(f"Category: {category}")
    
    for test_name, test_data in tests.items():
        df = pd.DataFrame(test_data["dataframe"])  # Convert items to DataFrame
        print(f"Test: {test_name} - Data Shape: {df.shape}")

        # Example analysis: Checking for missing values
#        print(df.isnull().sum())

        # You can perform further analyses here...


Category: Clinical
Test: PhQ-9 - Data Shape: (0, 0)
Test: DASS21 - Data Shape: (21, 3)
Test: DTDD - Data Shape: (0, 0)
Test: NPI - Data Shape: (0, 0)
Category: Social
Test: RSE - Data Shape: (0, 0)
Test: RWA - Data Shape: (0, 0)
Test: IRI index - Data Shape: (0, 0)
Category: Personality
Test: BIG 5 - Data Shape: (0, 0)
Test: HEXACO - Data Shape: (0, 0)
Test: NEO-PI - Data Shape: (0, 0)
